![prova](https://camo.githubusercontent.com/906e661107a3bc03104ca5d88336d1f4b0e80fdcac65efaf7904041d371c747f/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f6b657261732e696f2f696d672f6b657261732d6c6f676f2d323031382d6c617267652d313230302e706e67)

Keras is a high-level library leveraging TensorFlow as backend. It allows to build and experiment with ML models with an easy and flexible (?) interface.

API Docs (go to `tf.keras`): https://www.tensorflow.org/versions

Keras Guides (for specific API components): https://www.tensorflow.org/guide/keras

Tutorials (cover examples of basic usage): https://www.tensorflow.org/tutorials/keras

**Warning:** in general, it is better to use Keras whenever it is possible, since it exploits compiled computational graphs, making it way more efficent than TensorFlow (with eager execution activated, which is the default behaviour).

In [ ]:
import tensorflow as tf
from tensorflow import keras as K
import numpy as np
import seaborn as sns
import pandas as pd
sns.set_theme()

In [ ]:
K.__version__

In [ ]:
tf.__version__

In [ ]:
from sklearn.model_selection import train_test_split

(train_X, train_y), (test_X, test_y) = tf.keras.datasets.mnist.load_data(path='ds')
train_X, eval_X, train_y, eval_y = train_test_split(
    train_X.numpy(), train_y,
    test_size=0.15,
    shuffle=True,
    stratify=train_y
)

# Three interfaces for building a Model on Keras

Now we will see how to solve the same problem we've seen before with these three interfaces.

## Sequential API

Everything is wrapped within the ``K.Sequential()`` "box", and all the objects that you add to this box are considered layers. Note that, here, a layer is not necessarily a neural layer, but whatever applies a transformation to the output of the previous layer (or to the input data).

In general, you can consider the K.Sequential() interface as a wrapper for the pipeline which applies all the subsequent transformations to your input data, ending to the prediction of your model.

In [ ]:
seq_model = K.Sequential()

seq_model.add(K.layers.Flatten())
seq_model.add(K.layers.Rescaling(scale=1./255))
seq_model.add(K.layers.Dense(10, activation='softmax')) # Where's the input dimension?

### Functional API

In the Functional API, you explicitly create the pipeline we discussed before by performing subsequent applications of all the functions, __starting from a placeholder defining the input tensor__.

In [ ]:
inputs = K.Input(shape=(28, 28)) # Implicitly, the shape will be (None, 28, 28)
flattened = K.layers.Flatten()(inputs)
rescaled = K.layers.Rescaling(scale=1/255.)(flattened)
outputs = K.layers.Dense(units=10, activation='softmax')(rescaled)
fn_model = K.Model(inputs=inputs, outputs=outputs)
fn_model.summary()

## K.Model API

This is the best chance you have to exploit the synergy between Keras and TensorFlow. You can define your own custom model by inheriting from the class ``K.Model``.

In [ ]:
class SimpleLinearClassifier(K.Model):
    def __init__(self):
        super().__init__()
        self.flatten = K.layers.Flatten()
        self.dense = K.layers.Dense(units=10, activation='softmax')
    
    def call(self, inputs):
        x = self.flatten(inputs)
        x = x / 255.
        return self.dense(x)

model = SimpleLinearClassifier()

# Compiling the pipeline

As mentioned at the beginning, the greatest advantage of Keras over TensorFlow is that our processing pipeline is compiled on a static computational graph.

In [ ]:
help(model.compile)

In [ ]:
model.compile(
    optimizer=K.optimizers.Adam(learning_rate=0.1),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Model Training

Training a model with Keras is as simple as calling the `.fit` method.

In [ ]:
help(model.fit)

The ``callbacks`` parameter allows to add a set of functionalities which are called at specific times of the training loop.

In [ ]:
early_stopping = K.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=5,
    min_delta=0.0005,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    x=train_X, 
    y=train_y, 
    epochs=100, 
    batch_size=1000,
    validation_data=(eval_X, eval_y),
    callbacks=[early_stopping]
)

And we have the additional advantage of the training history, providing statistics of the training phase.

In [ ]:
list(history.history)

Questo è molto utile per poter visualizzare il comportamento del modello dopo il training.

In [ ]:
results = pd.DataFrame({
    'epoch': history.epoch,
    'loss': history.history['loss'],
    'val_loss': history.history['val_loss']
})
ax = sns.lineplot(x='epoch', y='value', hue='variable', data=pd.melt(results, ['epoch']))
ax.set(xlabel='epoch', ylabel='loss value')

In [ ]:
results = pd.DataFrame({
    'epoch': history.epoch,
    'loss': history.history['accuracy'],
    'val_loss': history.history['val_accuracy']
})
ax = sns.lineplot(x='epoch', y='value', hue='variable', data=pd.melt(results, ['epoch']))
ax.set(xlabel='epoch', ylabel='accuracy value')

In [ ]:
ax = sns.lineplot(x=history.epoch, y=history.history['accuracy'])
ax.set(xlabel='epoch', ylabel='accuracy')

Keras models expose the method `.evaluate`, which returns the results of the metrics on a given dataset.

In [ ]:
metrics = model.evaluate(test_X, test_y)
metrics # loss, accuracy

Instead, `.predict` returns the predictions of the model

In [ ]:
predictions = model.predict(test_X)
predictions

## Your turn!

Riprendiamo il problema dell'ultima volta, provate a implementare una rete neurale deep con Keras, e allenatela in GPU.
Il modello deve avere:

1. Un layer da 300 unità con attivazione `tanh`;
2. Un layer da 100 unità con attivazione `tanh`;
3. Un layer da 1 unità senza attivazione.

Questa volta, invece di utilizzare il metodo di standardizzazione di sklearn (per intenderci, lo `StandardScaler`), utilizzate il layer [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) di Keras. E' sufficiente, invece di chiamare la funzione `.fit` dello StandardScaler, che chiamiate la funzione `.adapt` del layer che create sui dati di training.

In [ ]:
(h__train_X, h_train_Y), (h_test_X, h_test_y) = tf.keras.datasets.boston_housing.load_data()

## Salvataggio e caricamento del modello

Ora che abbiamo un modello che potremmo anche voler riutilizzare, avere un metodo per il salvataggio e il caricamento è fondamentale.

La  `model serialization` può essere utile anche quando usate colab, visto che il tempo di utilizzo è limitato e si può disconnettere nel mezzo del processo e potreste voler riprendere il training in seguito.

In [ ]:
model.save('my_model')

In [ ]:
loaded_model = K.models.load_model('my_model')

Ci sono parecchie altre opzioni. Salvataggio in formato H5, salvare solo i pesi, salvare solo l'architettura del modello, etc...  
Guida alla serializzazione dei modelli: https://www.tensorflow.org/guide/keras/save_and_serialize

# Monitoring degli esperimenti con TensorBoard

Per comprendere ciò che avviene durante il training, è importante fare logging di valori, creare grafici e così via. 

**Tenere da parte un log di testo può sempre tornare utile, potreste voler creare visualizzazioni "custom".**

Esistono alcuni strumenti che facilitano il procedimento, tra cui TensorBoard, che fornisce un'interfaccia web da cui controllare l'andamento del training del nostro modello.

In [ ]:
# this is only needed in a notebook
%load_ext tensorboard 

In [ ]:
tensorboard_callback = K.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

In [ ]:
%tensorboard --logdir ./logs

In [ ]:
model = K.Sequential()
model.add(K.layers.Flatten())
model.add(K.layers.Rescaling(scale=1/255.))
#model.add(K.layers.Dense(1000, activation='relu')) # Nuovo layer non lineare
model.add(K.layers.Dense(200, activation='relu')) # Nuovo layer non lineare
model.add(K.layers.Dense(10, activation='softmax'))

model.compile(
    optimizer=K.optimizers.Adam(learning_rate=0.001),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(
    x=train_X, 
    y=train_y, 
    epochs=10000, 
    batch_size=1000,
    validation_data=(eval_X, eval_y),
    callbacks=[early_stopping, tensorboard_callback]
)

# Your Turn!

Dato lo split fornito di seguito, completate l'esercizio precedente aggiungendo anche il validation set e l'early stopping alla funzione `.fit`.

Opzionale: visualizzate il tutto anche con la TensorBoard!

In [ ]:
(h_train_X, h_train_y), (h_test_X, h_test_y) = tf.keras.datasets.boston_housing.load_data()

In [ ]:
import random

h_train = np.concatenate([h_train_X, h_train_y[:, np.newaxis]], axis=1)
random.seed(42)
random.shuffle(h_train)

h_train, h_eval = h_train[75:], h_train[:75]
h_train_X, h_train_y = h_train[:, :-1], h_train[:, -1]
h_eval_X, h_eval_y = h_eval[:, :-1], h_eval[:, -1]

In [ ]:
(h_train_X.shape, h_train_y.shape), (h_eval_X.shape, h_eval_y.shape)